In [ ]:
import tkinter as tk
from tkinter import messagebox
from tkinter import ttk
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
from priyangshu import rk2, rk4
from sympy import symbols,sympify

class EquationSolverApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Equation Solver App")

        self.equation_label = ttk.Label(root, text="Enter Equation:")
        self.equation_label.grid(row=0, column=0, padx=10, pady=5, sticky="W")

        self.equation_entry = ttk.Entry(root, width=30)
        self.equation_entry.grid(row=0, column=1, padx=10, pady=5)

        self.method_label = ttk.Label(root, text="Select Method:")
        self.method_label.grid(row=1, column=0, padx=10, pady=5, sticky="W")

        self.method_var = tk.StringVar()
        self.method_combobox = ttk.Combobox(root, textvariable=self.method_var, values=["rk2", "rk4", "fdm", "cdm", "bdm", "odeint"])
        self.method_combobox.grid(row=1, column=1, padx=10, pady=5)
        self.method_combobox.set("rk2")

        self.plot_checkbox_var = tk.BooleanVar()
        self.plot_checkbox = ttk.Checkbutton(root, text="Plot Graph", variable=self.plot_checkbox_var)
        self.plot_checkbox.grid(row=2, column=0, columnspan=2, pady=5)

        self.solve_button = ttk.Button(root, text="Solve", command=self.solve_equation)
        self.solve_button.grid(row=3, column=0, columnspan=2, pady=10)

    def solve_equation(self):
        equation = self.equation_entry.get()
        method = self.method_var.get()

        if method == "odeint":
            self.plot_solution(equation, method)
        else:
            try:
                # Attempt to solve the equation
                solution = self.solve_numeric(equation, method)
                self.plot_solution(equation, method, solution)
            except Exception as e:
                messagebox.showerror("Error", f"Error solving equation:\n{str(e)}")

    def solve_numeric(self, equation, method):
        x, y = symbols('x y')

        try:
        # Use sympify to handle the equation
            dydx = sympify(equation)
        except Exception as e:
            raise ValueError(f"Error parsing equation: {str(e)}")

        # Convert the evaluated expression to a Python function
        def f(y, x):
            return np.array([y[1], dydx.subs({x: y[0], y: y[1]})], dtype=float)

        t = np.linspace(0, 10, 1000)
        y0 = np.array([1.0, 0.0])

        if method == "rk2":
            solution = rk2(f, y0, t)
        elif method == "rk4":
            solution = rk4(f, y0, t)
        else:
            raise ValueError("Unsupported method selected")

        return solution

    def plot_solution(self, equation, method, solution=None):
        # Plot the solution graph
        if solution is None:
            messagebox.showinfo("Info", "No solution to plot.")
            return

        plt.figure(figsize=(8, 6))
        plt.plot(solution, label=f"{method} Solution")
        plt.title(f"Solution to {equation} using {method}")
        plt.xlabel("Time")
        plt.ylabel("Solution")
        plt.legend()

        if self.plot_checkbox_var.get():
            plt.show()

if __name__ == "__main__":
    root = tk.Tk()
    app = EquationSolverApp(root)
    root.mainloop()
